<a href="https://colab.research.google.com/github/aleks-haksly/ml-course-hse/blob/master/%D0%92%D0%B5%D0%B1%D0%B8%D0%BD%D0%B0%D1%80_11_%D0%B8%D1%8E%D0%BD%D1%8F_%D0%9D%D0%B5%D0%B9%D1%80%D0%BE%D1%81%D0%B5%D1%82%D0%B8_%D0%B4%D0%BB%D1%8F_%D1%82%D0%B0%D0%B1%D0%BB%D0%B8%D1%87%D0%BD%D1%8B%D1%85_%D0%B4%D0%B0%D0%BD%D0%BD%D1%8B%D1%85.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **ВЕБИНАР 11-06: Нейросети для табличных данных**

In [ ]:
import json
from datetime import datetime
import requests
import re
import os
import getpass
import numpy as np

#@title Установка библиотек для нейро-куратора

os.environ["UII_API_KEY"] = "bd5a0bfd-9385-408e-a466-0f648f63c0e7"
user_id_seed = np.random.randint(90000000001, 90000000100)

# **Табличные данные**

Табличные данные - это данные, представленные в виде таблицы, где строки обычно соответствуют записям (наблюдениям), а столбцы - признакам (атрибутам). Таблицы широко используются в анализе данных и машинном обучении.


## Типы данных в ячейках таблиц

Табличные данные могут включать в себя:
- **Числовые данные**: целые числа и вещественные числа (например, возраст, доход).
- **Категориальные данные**: текстовые значения, категории (например, пол, город).
- **Текстовые данные**: неструктурированные текстовые данные (например, отзывы, комментарии).
- **Даты и временные ряды**: даты, времена (например, дата регистрации, дата измерения).
- **Булевы данные**: значения истинности (например, да/нет, True/False).
- **Идентификаторы**: уникальные ключи или идентификаторы (например, номера записей).
- **Координаты**: географические координаты (например, широта и долгота).

## Основные этапы работы с табличными данными

Работа с табличными данными обычно включает несколько ключевых этапов:
1. **Чтение данных**: загрузка данных из файлов.
2. **Очистка данных**: обработка пропущенных значений, удаление дубликатов и аномалий.
3. **Анализ данных**: исследование распределений, выявление зависимостей и аномалий.
4. **Параметризация данных**: преобразование категориальных данных в числовые.
5. **Нормализация данных**: приведение данных к единому масштабу.

## Задачи с табличными данными


- **Регрессия**: предсказание непрерывного значения (например, цена автомобиля, доход, температура, уровень загрязнения воздуха).
  - Пример: Прогнозирование цены дома на основе его характеристик.
  - Пример: Прогнозирование уровня дохода на основе демографических данных.
  - Пример: Прогнозирование температуры воздуха на основе погодных условий.

- **Классификация**: предсказание категориальной метки (например, диагноз, пол, категория продукта, вид цветка).
  - Пример: Классификация типа цветка на основе его характеристик.
  - Пример: Определение диагноза пациента на основе его медицинских показателей.
  - Пример: Классификация отзывов на положительные и отрицательные на основе текста отзыва.



## Пример таблицы данных

In [ ]:
#@title Сервисные функции (ЗАПУСТИТЕ ЭТОТ КОД)
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

def create_dataframe():
    # Создадим таблицу данных
    data = {
        'ID': [1, 2, 3, 4, 5],
        'Имя': ['Иван', 'Анна', 'Олег', 'Ольга', 'Павел'],
        'Возраст': [25, 30, 35, 40, 45],
        'Пол': ['М', 'Ж', 'М', 'Ж', 'М'],
        'Город': ['Москва', 'Санкт-Петербург', 'Новосибирск', 'Екатеринбург', 'Казань'],
        'Доход': [50000, 60000, 70000, 80000, 90000],
        'Дата регистрации': ['2022-01-15', '2021-12-01', '2020-11-20', '2019-10-05', '2018-09-10'],
        'Отзыв': ['Отличный сервис', 'Очень доволен обслуживанием', 'Все прошло хорошо, рекомендую', 'Быстро и качественно', 'Сервис на высшем уровне'],
        'Активен': ['Да', 'Нет', 'Да', 'Нет', 'Да'],
        'Широта': [55.7558, 59.9343, 55.0084, 56.8389, 55.7963],
        'Долгота': [37.6173, 30.3351, 82.9357, 60.6057, 49.1064]
    }

    df = pd.DataFrame(data)
    return df

def process_date(df):
    df['Дата регистрации'] = pd.to_datetime(df['Дата регистрации'])
    first_date = df['Дата регистрации'].min()
    df['Дни с регистрации'] = (df['Дата регистрации'] - first_date).dt.days
    df = df.drop(columns=['Дата регистрации'])
    return df

def scale_dataframe(df):
    scaler = MinMaxScaler()
    df[['Возраст', 'Доход', 'Широта', 'Долгота', 'Дни с регистрации']] = scaler.fit_transform(df[['Возраст', 'Доход', 'Широта', 'Долгота', 'Дни с регистрации']])
    return df

def transform_dataframe(df):
    # Преобразование пола в one-hot encoding
    df['Пол'] = df['Пол'].map({'М': [1, 0], 'Ж': [0, 1]})

    # Преобразование городов в числовые метки
    city_labels = {'Москва': 1, 'Санкт-Петербург': 2, 'Новосибирск': 3, 'Екатеринбург': 4, 'Казань': 5}
    df['Город'] = df['Город'].map(city_labels)

    # Преобразование имен в числовые метки
    name_labels = {'Иван': 1, 'Анна': 2, 'Олег': 3, 'Ольга': 4, 'Павел': 5}
    df['Имя'] = df['Имя'].map(name_labels)

    # Преобразование столбца 'Активен' в числовой формат
    df['Активен'] = df['Активен'].map({'Да': 1, 'Нет': 0})

    return df

def encode_reviews(df):
    # Создание словаря из уникальных слов
    all_words = set(word for review in df['Отзыв'] for word in review.split())
    word_to_index = {word: i for i, word in enumerate(all_words)}

    # Преобразование отзывов в one-hot encoding векторы
    def review_to_ohe(review, word_to_index):
        ohe_vector = [0] * len(word_to_index)
        for word in review.split():
            ohe_vector[word_to_index[word]] = 1
        return ohe_vector

    df['Отзыв_ОХЕ'] = df['Отзыв'].apply(lambda x: review_to_ohe(x, word_to_index))
    df = df.drop(columns=['Отзыв'])  # Удаление исходного столбца 'Отзыв'
    return df

In [ ]:
# Пример таблицы с различными данными
df = create_dataframe()
df

,ID,Имя,Возраст,Пол,Город,Доход,Дата регистрации,Отзыв,Активен,Широта,Долгота
0,1,Иван,25,М,Москва,50000,2022-01-15,Отличный сервис,Да,55.7558,37.6173
1,2,Анна,30,Ж,Санкт-Петербург,60000,2021-12-01,Очень доволен обслуживанием,Нет,59.9343,30.3351
2,3,Олег,35,М,Новосибирск,70000,2020-11-20,"Все прошло хорошо, рекомендую",Да,55.0084,82.9357
3,4,Ольга,40,Ж,Екатеринбург,80000,2019-10-05,Быстро и качественно,Нет,56.8389,60.6057
4,5,Павел,45,М,Казань,90000,2018-09-10,Сервис на высшем уровне,Да,55.7963,49.1064


## Пример преобразования даты

In [ ]:
# Преобразование даты регистрации в количество дней с первой даты
df = process_date(df)
df

,ID,Имя,Возраст,Пол,Город,Доход,Отзыв,Активен,Широта,Долгота,Дни с регистрации
0,1,Иван,25,М,Москва,50000,Отличный сервис,Да,55.7558,37.6173,1223
1,2,Анна,30,Ж,Санкт-Петербург,60000,Очень доволен обслуживанием,Нет,59.9343,30.3351,1178
2,3,Олег,35,М,Новосибирск,70000,"Все прошло хорошо, рекомендую",Да,55.0084,82.9357,802
3,4,Ольга,40,Ж,Екатеринбург,80000,Быстро и качественно,Нет,56.8389,60.6057,390
4,5,Павел,45,М,Казань,90000,Сервис на высшем уровне,Да,55.7963,49.1064,0


## Пример нормализации данных

In [ ]:
# Пример нормализации столбцов Возраст, Доход, Активен, Широта, Долгота, Дни с регистрации (от 0 до 1)
df = scale_dataframe(df)
df

,ID,Имя,Возраст,Пол,Город,Доход,Отзыв,Активен,Широта,Долгота,Дни с регистрации
0,1,Иван,0.00,М,Москва,0.00,Отличный сервис,Да,0.151729,0.138443,1.000000
1,2,Анна,0.25,Ж,Санкт-Петербург,0.25,Очень доволен обслуживанием,Нет,1.000000,0.000000,0.963205
2,3,Олег,0.50,М,Новосибирск,0.50,"Все прошло хорошо, рекомендую",Да,0.000000,1.000000,0.655765
3,4,Ольга,0.75,Ж,Екатеринбург,0.75,Быстро и качественно,Нет,0.371607,0.575480,0.318888
4,5,Павел,1.00,М,Казань,1.00,Сервис на высшем уровне,Да,0.159950,0.356865,0.000000


## Примеры параметризации данных

In [ ]:
# Пример параметризации столбцов: Имя (метки), Пол(OHE), Город(метки), Активен(метки)
df = transform_dataframe(df)
df

,ID,Имя,Возраст,Пол,Город,Доход,Отзыв,Активен,Широта,Долгота,Дни с регистрации
0,1,1,0.00,"[1, 0]",1,0.00,Отличный сервис,1,0.151729,0.138443,1.000000
1,2,2,0.25,"[0, 1]",2,0.25,Очень доволен обслуживанием,0,1.000000,0.000000,0.963205
2,3,3,0.50,"[1, 0]",3,0.50,"Все прошло хорошо, рекомендую",1,0.000000,1.000000,0.655765
3,4,4,0.75,"[0, 1]",4,0.75,Быстро и качественно,0,0.371607,0.575480,0.318888
4,5,5,1.00,"[1, 0]",5,1.00,Сервис на высшем уровне,1,0.159950,0.356865,0.000000


In [ ]:
# Пример преобразования столбца: Отзыв в OHE вектор
df = encode_reviews(df)
df

,ID,Имя,Возраст,Пол,Город,Доход,Активен,Широта,Долгота,Дни с регистрации,Отзыв_ОХЕ
0,1,1,0.00,"[1, 0]",1,0.00,1,0.151729,0.138443,1.000000,"[1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]"
1,2,2,0.25,"[0, 1]",2,0.25,0,1.000000,0.000000,0.963205,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0]"
2,3,3,0.50,"[1, 0]",3,0.50,1,0.000000,1.000000,0.655765,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1]"
3,4,4,0.75,"[0, 1]",4,0.75,0,0.371607,0.575480,0.318888,"[0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]"
4,5,5,1.00,"[1, 0]",5,1.00,1,0.159950,0.356865,0.000000,"[0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0]"


## Основные проблемы при работе с табличными данными

1. **Пропущенные данные (Missing Data)**
   - Пропущенные значения могут существенно повлиять на качество модели. Для их обработки можно использовать методы удаления или заполнения пропусков (импутация).

2. **Шум в данных (Noise in Data)**
   - Шумные данные могут снизить точность модели. Методы выявления и удаления выбросов (outliers) помогают улучшить качество данных.

3. **Смещение и дисбаланс классов (Class Imbalance)**
   - В задачах классификации часто встречается дисбаланс классов, когда один класс представлен значительно чаще других. Это может привести к тому, что модель будет плохо предсказывать менее представленные классы. Методы балансировки, такие как oversampling и undersampling, могут помочь решить эту проблему.

4. **Мультиколлинеарность (Multicollinearity)**
   - В задачах регрессии мультиколлинеарность возникает, когда независимые переменные сильно коррелируют друг с другом, что может усложнить интерпретацию модели. Методы уменьшения размерности, такие как PCA, или удаление коррелирующих признаков могут помочь.

5. **Высокая размерность данных (High Dimensionality)**
   - Множество признаков может усложнить обучение модели и привести к переобучению. Методы уменьшения размерности, такие как PCA или отбор признаков, могут снизить размерность данных.

6. **Отсутствие стандартных масштабов (Lack of Standardization)**
   - Различные признаки могут иметь разные масштабы, что может привести к проблемам при обучении модели. Масштабирование данных (например, с помощью стандартизации или нормализации) помогает решить эту проблему.

7. **Переполненные данные (Overfitting)**
   - Модель может слишком хорошо подстроиться под тренировочные данные и плохо обобщаться на новые данные. Регуляризация и кросс-валидация могут помочь уменьшить переобучение.

8. **Недостаточные данные (Insufficient Data)**
   - Недостаток данных может привести к недообучению модели. Методы увеличения данных (data augmentation) или сбор дополнительных данных могут быть полезны.


In [ ]:
# @title Задать вопрос нейро-куратору

question = "В какие типы данных нужно преобразовать табличную информацию, чтобы ее смогла обработать нейронная сеть?" #@param {type:"string"}
url = "http://31.129.102.77:80/curatorv1/qa"
headers = {"Content-Type": "application/json",
    "Authorization": os.environ["UII_API_KEY"]
    }
data = {
    "question": question,
    "user_id": user_id_seed,
    "topic": ''
}

response = requests.post(url, headers=headers, data=json.dumps(data))
print(json.loads(response.text))

Для того чтобы нейронная сеть могла обработать табличную информацию, необходимо преобразовать данные в числовой формат. Нейронные сети работают с числами, поэтому все категориальные и текстовые данные из таблицы нужно преобразовать в числовой вид. 

Вот несколько типов данных, в которые можно преобразовать табличную информацию для обработки нейронной сетью:

1. **One-Hot Encoding**: Для категориальных переменных, таких как тип дома или метод транспортировки, можно использовать метод One-Hot Encoding. При этом каждая категория преобразуется в бинарный вектор, где все элементы равны нулю, кроме одного, соответствующего категории.

2. **Масштабирование**: Для числовых переменных важно провести масштабирование данных, чтобы они находились в одном диапазоне значений. Например, можно использовать стандартизацию (Standard Scaling) или нормализацию (Min-Max Scaling).

3. **Текстовые данные**: Если в таблице есть текстовые данные, их можно преобразовать с помощью методов векторизации, таких как

# **1. Задача регрессии: Предсказание стоимости автомобиля**

In [ ]:
#@title Загрузка необходимых библиотек (ЗАПУСТИТЕ ЭТОТ КОД)
!pip install h2o
import pandas as pd
import numpy as np
import h2o
from h2o.automl import H2OAutoML
import gdown
import warnings
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import mean_absolute_error
from IPython.display import clear_output
import pandas as pd
import matplotlib.pyplot as plt

# Игнорирование предупреждений H2ODependencyWarning
warnings.filterwarnings("ignore", category=h2o.exceptions.H2ODependencyWarning)

# Инициализация H2O
h2o.init()
clear_output()

In [ ]:
#@title Сервисные функции (ЗАПУСТИТЕ ЭТОТ КОД)
#@title Функции загрузки данных (ЗАПУСТИТЕ ЭТОТ КОД)
def load_data(file_path):
    df = pd.read_csv(file_path)
    df.drop_duplicates(inplace=True)
    df.dropna(inplace=True)
    return df

#@title Функция для преобразования категориальных признаков (ЗАПУСТИТЕ ЭТОТ КОД)
def encode_categorical_features(df, categorical_columns):
    encoders = {}
    for col in categorical_columns:
        encoders[col] = LabelEncoder()
        df[col] = encoders[col].fit_transform(df[col])
    return df, encoders

#@title Функция для нормализации данных (ЗАПУСТИТЕ ЭТОТ КОД)
def normalize_data(df, columns_to_normalize, target_column):
    scalers = {}

    # Нормализация данных
    for col in columns_to_normalize:
        scalers[col] = MinMaxScaler()
        df[col] = scalers[col].fit_transform(df[[col]])

    # Нормализация целевого значения
    scalers[target_column] = MinMaxScaler()
    df[target_column] = scalers[target_column].fit_transform(df[[target_column]])

    return df, scalers

#@title Функция для преобразования текстовых данных в числовые (ЗАПУСТИТЕ ЭТОТ КОД)
def encode_text_data(df, column, max_features):
    vectorizer = CountVectorizer(max_features=max_features)
    encoded = vectorizer.fit_transform(df[column]).toarray()
    encoded_df = pd.DataFrame(encoded, columns=vectorizer.get_feature_names_out())
    return encoded_df, vectorizer

#@title Функция предобработки данных (ЗАПУСТИТЕ ЭТОТ КОД)
def preprocess_data(df):
    # Переименование столбца 'mark' в 'mark_'
    df.rename(columns={'mark': 'mark_'}, inplace=True)

    # Преобразование категориальных признаков
    categorical_columns = ['mark_', 'body', 'kpp', 'fuel']
    df, encoders = encode_categorical_features(df, categorical_columns)

    # Преобразование текстового признака 'model'
    text_df, vectorizer = encode_text_data(df, 'model', max_features=1000)

    # Удаление оригинального текстового столбца 'model'
    df.drop(columns=['model'], inplace=True)

    # Объединение всех данных
    df = pd.concat([df.reset_index(drop=True), text_df.reset_index(drop=True)], axis=1)

    return df, encoders, vectorizer

#@title Функция для преподготовки данных к нормализации (ЗАПУСТИТЕ ЭТОТ КОД)
def prepare_and_normalize_data(df, target_column, columns_to_normalize):
    # Назначение целевой переменной
    y = df[target_column].values

    # Удаление столбца с целевой переменной из данных
    df.drop(columns=[target_column], inplace=True)

    # Назначение обучающих данных
    columns = df.columns.tolist()
    X = df.values

    # Нормализация данных перед разделением
    df_combined = pd.concat([pd.DataFrame(X, columns=columns), pd.Series(y, name=target_column)], axis=1)
    df_combined, scalers = normalize_data(df_combined, columns_to_normalize, target_column)

    return df_combined, scalers, columns

#@title Функция для создания и обучения модели (ЗАПУСТИТЕ ЭТОТ КОД)
def train_model(X_train, y_train, X_val, y_val, max_models, seed, max_runtime_secs, stopping_metric):
    train = h2o.H2OFrame(pd.concat([pd.DataFrame(X_train, columns=columns), pd.Series(y_train, name='price')], axis=1))
    valid = h2o.H2OFrame(pd.concat([pd.DataFrame(X_val, columns=columns), pd.Series(y_val, name='price')], axis=1))

    aml = H2OAutoML(
        max_models=max_models,
        seed=seed,
        max_runtime_secs=max_runtime_secs,
        stopping_metric=stopping_metric
    )
    aml.train(x=columns, y='price', training_frame=train, validation_frame=valid)

    return aml

#@title Функция для предсказания и оценки модели (ЗАПУСТИТЕ ЭТОТ КОД)
def evaluate_model(model, X_test, y_test):
    test = h2o.H2OFrame(pd.DataFrame(X_test, columns=columns))
    y_pred = model.predict(test).as_data_frame().values.flatten()
    mae = mean_absolute_error(y_test, y_pred)
    average_price = y_test.mean()
    percentage_error = (mae / average_price) * 100
    print(f"Средняя абсолютная ошибка на тестовой выборке: {int(mae)} рублей.")
    print(f"Процент ошибки относительно средней цены: {percentage_error:.2f}%")
    return y_pred, mae

#@title Функция для предсказания и оценки модели с нормализацией (ЗАПУСТИТЕ ЭТОТ КОД)
def evaluate_model_normalized(model, X_test, y_test, y_scaler):
    test = h2o.H2OFrame(pd.DataFrame(X_test, columns=columns))
    y_pred = model.predict(test).as_data_frame().values.flatten()
    y_test = y_scaler.inverse_transform(y_test.reshape(-1, 1)).flatten()
    y_pred = y_scaler.inverse_transform(y_pred.reshape(-1, 1)).flatten()
    mae = mean_absolute_error(y_test, y_pred)
    average_price = y_test.mean()
    percentage_error = (mae / average_price) * 100
    print(f"Средняя абсолютная ошибка на тестовой выборке: {int(mae)} рублей.")
    print(f"Процент ошибки относительно средней цены: {percentage_error:.2f}%")
    return y_pred, mae

**ОПИСАНИЕ ДАННЫХ**

В этой таблице содержатся данные о различных автомобилях. Таблица состоит из 10 колонок и 70119 строк. Вот описание каждой колонки:

- **mark**: Марка автомобиля.
- **model**: Модель автомобиля.
- **price**: Цена автомобиля в рублях.
- **year**: Год выпуска автомобиля.
- **mileage**: Пробег автомобиля в километрах.
- **body**: Тип кузова автомобиля.
- **kpp**: Тип коробки передач автомобиля.
- **fuel**: Тип топлива, используемого автомобилем.
- **volume**: Объем двигателя в литрах.
- **power**: Мощность двигателя в лошадиных силах.

Целевое значение, которое мы будем предсказывать, это **price** (цена автомобиля).



In [ ]:
# Загрузка данных
gdown.download('https://storage.yandexcloud.net/aiueducation/Content/base/l10/cars_new.csv', None, quiet=True)
# Чтение файла базы данных
df = load_data('/content/cars_new.csv')
df

,mark,model,price,year,mileage,body,kpp,fuel,volume,power
0,kia,cerato,996000,2018,28000,седан,автомат,бензин,2.0,150.0
1,daewoo,nexia 1 поколение [2-й рестайлинг],140200,2012,60500,седан,механика,бензин,1.5,80.0
2,suzuki,jimny 3 поколение [рестайлинг],750000,2011,29000,внедорожник,автомат,бензин,1.3,85.0
3,bmw,x1 18 e84 [рестайлинг],970000,2014,49500,кроссовер,автомат,бензин,2.0,150.0
4,chevrolet,lacetti 1 поколение,205000,2007,151445,седан,механика,бензин,1.4,95.0
...,...,...,...,...,...,...,...,...,...,...
70114,skoda,fabia 5j [рестайлинг],310000,2011,165000,хетчбэк,механика,бензин,1.2,70.0
70115,ford,focus 3 поколение,400000,2011,125000,хетчбэк,механика,бензин,1.6,105.0
70116,mazda,cx-5 1 поколение,1140000,2013,89981,кроссовер,автомат,бензин,2.0,150.0
70117,toyota,bb 2 поколение,550550,2006,183000,минивэн,автомат,бензин,1.3,92.0


#### **Эксперимент 1** (используем только числовые ячейки: year, mileage, volume, power)

In [ ]:
# Удаление столбцов, не участвующих в обучении модели
df.drop(columns=['mark', 'model', 'body', 'kpp', 'fuel'], inplace=True)
df

,price,year,mileage,volume,power
0,996000,2018,28000,2.0,150.0
1,140200,2012,60500,1.5,80.0
2,750000,2011,29000,1.3,85.0
3,970000,2014,49500,2.0,150.0
4,205000,2007,151445,1.4,95.0
...,...,...,...,...,...
70114,310000,2011,165000,1.2,70.0
70115,400000,2011,125000,1.6,105.0
70116,1140000,2013,89981,2.0,150.0
70117,550550,2006,183000,1.3,92.0


In [ ]:
# Определение столбцов для обучения и целевой переменной
columns = ['year', 'mileage', 'volume', 'power']
X = df[columns].values
y = df['price'].values

# Разделение данных на обучающую, валидационную и тестовую выборки
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.1, random_state=42)  # 90% для обучения, 10% для временной выборки
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)  # 5% для валидации и 5% для тестирования

# Вывод форм выборок
print("Форма обучающей выборки X_train:", X_train.shape)
print("Форма валидационной выборки X_val:", X_val.shape)
print("Форма тестовой выборки X_test:", X_test.shape)
print("Форма целевой обучающей выборки y_train:", y_train.shape)
print("Форма целевой валидационной выборки y_val:", y_val.shape)
print("Форма целевой тестовой выборки y_test:", y_test.shape)

Форма обучающей выборки X_train: (63100, 4)
Форма валидационной выборки X_val: (3506, 4)
Форма тестовой выборки X_test: (3506, 4)
Форма целевой обучающей выборки y_train: (63100,)
Форма целевой валидационной выборки y_val: (3506,)
Форма целевой тестовой выборки y_test: (3506,)


In [ ]:
# Обучение модели
model = train_model(
    X_train,  # Обучающая выборка
    y_train,  # Целевая переменная для обучения
    X_val,  # Валидационная выборка
    y_val,  # Целевая переменная для валидации
    max_models=20,  # Количество моделей
    seed=42,  # Зерно для воспроизводимости
    max_runtime_secs=1200,  # Максимальное время выполнения в секундах
    stopping_metric='mae'  # Метрика остановки
)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
AutoML progress: |
08:34:38.930: User specified a validation frame with cross-validation still enabled. Please note that the models will still be validated using cross-validation only, the validation frame will be used to provide purely informative validation metrics on the trained models.

███████████████████████████████████████████████████████████████| (done) 100%


In [ ]:
# Оценка модели
y_pred, mae = evaluate_model(model, X_test, y_test)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
xgboost prediction progress: |███████████████████████████████████████████████████| (done) 100%
Средняя абсолютная ошибка на тестовой выборке: 93315 рублей.
Процент ошибки относительно средней цены: 17.73%


#### **Эксперимент 2** (используем только числовые ячейки: year, mileage, volume, power с нормализацией)

In [ ]:
# Удаление столбцов, не участвующих в обучении модели
df.drop(columns=['mark', 'model', 'body', 'kpp', 'fuel'], inplace=True)
df

,price,year,mileage,volume,power
0,996000,2018,28000,2.0,150.0
1,140200,2012,60500,1.5,80.0
2,750000,2011,29000,1.3,85.0
3,970000,2014,49500,2.0,150.0
4,205000,2007,151445,1.4,95.0
...,...,...,...,...,...
70114,310000,2011,165000,1.2,70.0
70115,400000,2011,125000,1.6,105.0
70116,1140000,2013,89981,2.0,150.0
70117,550550,2006,183000,1.3,92.0


In [ ]:
# Нормализуем данные
columns_to_normalize = ['year', 'mileage', 'volume', 'power']
df, scalers, columns = prepare_and_normalize_data(df, 'price', columns_to_normalize)
df

,year,mileage,volume,power,price
0,0.985714,0.027999,0.175258,0.100072,0.040026
1,0.900000,0.060499,0.123711,0.049676,0.005073
2,0.885714,0.028999,0.103093,0.053276,0.029979
3,0.928571,0.049499,0.175258,0.100072,0.038964
4,0.828571,0.151444,0.113402,0.060475,0.007719
...,...,...,...,...,...
70107,0.885714,0.164999,0.092784,0.042477,0.012008
70108,0.885714,0.124999,0.134021,0.067675,0.015684
70109,0.914286,0.089980,0.175258,0.100072,0.045908
70110,0.814286,0.182999,0.103093,0.058315,0.021833


In [ ]:
# Определение столбцов для обучения и целевой переменной
columns = ['year', 'mileage', 'volume', 'power']
X = df[columns].values
y = df['price'].values

# Разделение данных на обучающую, валидационную и тестовую выборки
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.1, random_state=42)  # 90% для обучения, 10% для временной выборки
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)  # 5% для валидации и 5% для тестирования

# Вывод форм выборок
print("Форма обучающей выборки X_train:", X_train.shape)
print("Форма валидационной выборки X_val:", X_val.shape)
print("Форма тестовой выборки X_test:", X_test.shape)
print("Форма целевой обучающей выборки y_train:", y_train.shape)
print("Форма целевой валидационной выборки y_val:", y_val.shape)
print("Форма целевой тестовой выборки y_test:", y_test.shape)

Форма обучающей выборки X_train: (63100, 4)
Форма валидационной выборки X_val: (3506, 4)
Форма тестовой выборки X_test: (3506, 4)
Форма целевой обучающей выборки y_train: (63100,)
Форма целевой валидационной выборки y_val: (3506,)
Форма целевой тестовой выборки y_test: (3506,)


In [ ]:
# Обучение модели
model = train_model(
    X_train,  # Обучающая выборка
    y_train,  # Целевая переменная для обучения
    X_val,  # Валидационная выборка
    y_val,  # Целевая переменная для валидации
    max_models=20,  # Количество моделей
    seed=42,  # Зерно для воспроизводимости
    max_runtime_secs=1200,  # Максимальное время выполнения в секундах
    stopping_metric='mae'  # Метрика остановки
)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
AutoML progress: |
09:05:03.549: User specified a validation frame with cross-validation still enabled. Please note that the models will still be validated using cross-validation only, the validation frame will be used to provide purely informative validation metrics on the trained models.

███████████████████████████████████████████████████████████████| (done) 100%


In [ ]:
# Оценка модели
y_pred, mae = evaluate_model_normalized(model, X_test, y_test, scalers['price'])

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
xgboost prediction progress: |███████████████████████████████████████████████████| (done) 100%
Средняя абсолютная ошибка на тестовой выборке: 94761 рублей.
Процент ошибки относительно средней цены: 18.01%


#### **Эксперимент 3** (используем числовые ячейки: year, mileage, volume, power и категориальные ячейки: mark, body, kpp, fuel)

In [ ]:
# Удаление столбцов, не участвующих в обучении модели
df.drop(columns=['model'], inplace=True)
df

,mark,price,year,mileage,body,kpp,fuel,volume,power
0,kia,996000,2018,28000,седан,автомат,бензин,2.0,150.0
1,daewoo,140200,2012,60500,седан,механика,бензин,1.5,80.0
2,suzuki,750000,2011,29000,внедорожник,автомат,бензин,1.3,85.0
3,bmw,970000,2014,49500,кроссовер,автомат,бензин,2.0,150.0
4,chevrolet,205000,2007,151445,седан,механика,бензин,1.4,95.0
...,...,...,...,...,...,...,...,...,...
70114,skoda,310000,2011,165000,хетчбэк,механика,бензин,1.2,70.0
70115,ford,400000,2011,125000,хетчбэк,механика,бензин,1.6,105.0
70116,mazda,1140000,2013,89981,кроссовер,автомат,бензин,2.0,150.0
70117,toyota,550550,2006,183000,минивэн,автомат,бензин,1.3,92.0


In [ ]:
# Преобразование категориальных признаков
categorical_columns = ['mark', 'body', 'kpp', 'fuel']
df, encoders = encode_categorical_features(df, categorical_columns)
df

,mark,price,year,mileage,body,kpp,fuel,volume,power
0,8,996000,2018,28000,10,0,0,2.0,150.0
1,4,140200,2012,60500,10,2,0,1.5,80.0
2,18,750000,2011,29000,0,0,0,1.3,85.0
3,1,970000,2014,49500,2,0,0,2.0,150.0
4,3,205000,2007,151445,10,2,0,1.4,95.0
...,...,...,...,...,...,...,...,...,...
70114,16,310000,2011,165000,15,2,0,1.2,70.0
70115,5,400000,2011,125000,15,2,0,1.6,105.0
70116,9,1140000,2013,89981,2,0,0,2.0,150.0
70117,19,550550,2006,183000,7,0,0,1.3,92.0


In [ ]:
# Определение столбцов для обучения и целевой переменной
columns = ['year', 'mileage', 'volume', 'power'] + categorical_columns
X = df[columns].values
y = df['price'].values

# Разделение данных на обучающую, валидационную и тестовую выборки
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.1, random_state=42)  # 90% для обучения, 10% для временной выборки
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)  # 5% для валидации и 5% для тестирования

# Вывод шейпов выборок
print("Форма обучающей выборки X_train:", X_train.shape)
print("Форма валидационной выборки X_val:", X_val.shape)
print("Форма тестовой выборки X_test:", X_test.shape)
print("Форма целевой обучающей выборки y_train:", y_train.shape)
print("Форма целевой валидационной выборки y_val:", y_val.shape)
print("Форма целевой тестовой выборки y_test:", y_test.shape)

Форма обучающей выборки X_train: (63100, 8)
Форма валидационной выборки X_val: (3506, 8)
Форма тестовой выборки X_test: (3506, 8)
Форма целевой обучающей выборки y_train: (63100,)
Форма целевой валидационной выборки y_val: (3506,)
Форма целевой тестовой выборки y_test: (3506,)


In [ ]:
# Обучение модели
model = train_model(
    X_train,  # Обучающая выборка
    y_train,  # Целевая переменная для обучения
    X_val,  # Валидационная выборка
    y_val,  # Целевая переменная для валидации
    max_models=20,  # Количество моделей
    seed=42,  # Зерно для воспроизводимости
    max_runtime_secs=1200,  # Максимальное время выполнения в секундах
    stopping_metric='mae'  # Метрика остановки
)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
AutoML progress: |
09:18:46.617: User specified a validation frame with cross-validation still enabled. Please note that the models will still be validated using cross-validation only, the validation frame will be used to provide purely informative validation metrics on the trained models.

███████████████████████████████████████████████████████████████| (done) 100%


In [ ]:
# Оценка модели
y_pred, mae = evaluate_model(model, X_test, y_test)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
gbm prediction progress: |███████████████████████████████████████████████████████| (done) 100%
Средняя абсолютная ошибка на тестовой выборке: 72058 рублей.
Процент ошибки относительно средней цены: 13.69%


#### **Эксперимент 4** (используем числовые ячейки: year, mileage, volume, power и категориальные ячейки: mark, body, kpp, fuel с нормализацией)

In [ ]:
# Удаление столбцов, не участвующих в обучении модели
df.drop(columns=['model'], inplace=True)
df

,mark,price,year,mileage,body,kpp,fuel,volume,power
0,kia,996000,2018,28000,седан,автомат,бензин,2.0,150.0
1,daewoo,140200,2012,60500,седан,механика,бензин,1.5,80.0
2,suzuki,750000,2011,29000,внедорожник,автомат,бензин,1.3,85.0
3,bmw,970000,2014,49500,кроссовер,автомат,бензин,2.0,150.0
4,chevrolet,205000,2007,151445,седан,механика,бензин,1.4,95.0
...,...,...,...,...,...,...,...,...,...
70114,skoda,310000,2011,165000,хетчбэк,механика,бензин,1.2,70.0
70115,ford,400000,2011,125000,хетчбэк,механика,бензин,1.6,105.0
70116,mazda,1140000,2013,89981,кроссовер,автомат,бензин,2.0,150.0
70117,toyota,550550,2006,183000,минивэн,автомат,бензин,1.3,92.0


In [ ]:
# Преобразование категориальных признаков
categorical_columns = ['mark', 'body', 'kpp', 'fuel']
df, encoders = encode_categorical_features(df, categorical_columns)
df

,mark,price,year,mileage,body,kpp,fuel,volume,power
0,8,996000,2018,28000,10,0,0,2.0,150.0
1,4,140200,2012,60500,10,2,0,1.5,80.0
2,18,750000,2011,29000,0,0,0,1.3,85.0
3,1,970000,2014,49500,2,0,0,2.0,150.0
4,3,205000,2007,151445,10,2,0,1.4,95.0
...,...,...,...,...,...,...,...,...,...
70114,16,310000,2011,165000,15,2,0,1.2,70.0
70115,5,400000,2011,125000,15,2,0,1.6,105.0
70116,9,1140000,2013,89981,2,0,0,2.0,150.0
70117,19,550550,2006,183000,7,0,0,1.3,92.0


In [ ]:
# Нормализуем данные
columns_to_normalize = ['year', 'mileage', 'volume', 'power', 'mark', 'body', 'kpp', 'fuel']
df, scalers, columns = prepare_and_normalize_data(df, 'price', columns_to_normalize)
df

,mark,year,mileage,body,kpp,fuel,volume,power,price
0,0.40,0.985714,0.027999,0.666667,0.000000,0.0,0.175258,0.100072,0.040026
1,0.20,0.900000,0.060499,0.666667,0.666667,0.0,0.123711,0.049676,0.005073
2,0.90,0.885714,0.028999,0.000000,0.000000,0.0,0.103093,0.053276,0.029979
3,0.05,0.928571,0.049499,0.133333,0.000000,0.0,0.175258,0.100072,0.038964
4,0.15,0.828571,0.151444,0.666667,0.666667,0.0,0.113402,0.060475,0.007719
...,...,...,...,...,...,...,...,...,...
70107,0.80,0.885714,0.164999,1.000000,0.666667,0.0,0.092784,0.042477,0.012008
70108,0.25,0.885714,0.124999,1.000000,0.666667,0.0,0.134021,0.067675,0.015684
70109,0.45,0.914286,0.089980,0.133333,0.000000,0.0,0.175258,0.100072,0.045908
70110,0.95,0.814286,0.182999,0.466667,0.000000,0.0,0.103093,0.058315,0.021833


In [ ]:
# Определение столбцов для обучения и целевой переменной
columns = ['year', 'mileage', 'volume', 'power'] + categorical_columns
X = df[columns].values
y = df['price'].values

# Разделение данных на обучающую, валидационную и тестовую выборки
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.1, random_state=42)  # 90% для обучения, 10% для временной выборки
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)  # 5% для валидации и 5% для тестирования

# Вывод шейпов выборок
print("Форма обучающей выборки X_train:", X_train.shape)
print("Форма валидационной выборки X_val:", X_val.shape)
print("Форма тестовой выборки X_test:", X_test.shape)
print("Форма целевой обучающей выборки y_train:", y_train.shape)
print("Форма целевой валидационной выборки y_val:", y_val.shape)
print("Форма целевой тестовой выборки y_test:", y_test.shape)

Форма обучающей выборки X_train: (63100, 8)
Форма валидационной выборки X_val: (3506, 8)
Форма тестовой выборки X_test: (3506, 8)
Форма целевой обучающей выборки y_train: (63100,)
Форма целевой валидационной выборки y_val: (3506,)
Форма целевой тестовой выборки y_test: (3506,)


In [ ]:
# Обучение модели
model = train_model(
    X_train,  # Обучающая выборка
    y_train,  # Целевая переменная для обучения
    X_val,  # Валидационная выборка
    y_val,  # Целевая переменная для валидации
    max_models=20,  # Количество моделей
    seed=42,  # Зерно для воспроизводимости
    max_runtime_secs=1200,  # Максимальное время выполнения в секундах
    stopping_metric='mae'  # Метрика остановки
)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
AutoML progress: |
09:44:44.856: User specified a validation frame with cross-validation still enabled. Please note that the models will still be validated using cross-validation only, the validation frame will be used to provide purely informative validation metrics on the trained models.

███████████████████████████████████████████████████████████████| (done) 100%


In [ ]:
# Оценка модели
y_pred, mae = evaluate_model_normalized(model, X_test, y_test, scalers['price'])

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
gbm prediction progress: |███████████████████████████████████████████████████████| (done) 100%
Средняя абсолютная ошибка на тестовой выборке: 70646 рублей.
Процент ошибки относительно средней цены: 13.43%


#### **Эксперимент 5** (используем числовые ячейки: year, mileage, volume, power категориальные ячейки: mark, body, kpp, fuel тестовую ячейку: model)

In [ ]:
# Запускаем функцию предобработки данных + добавляем вектор текста длинной 1000 параметров
df, encoders, vectorizer = preprocess_data(df)
df

,mark_,price,year,mileage,body,kpp,fuel,volume,power,100,...,z52,zafira,класс,поколение,рестайлинг,с23,с3,серия,сс,х80
0,8,996000,2018,28000,10,0,0,2.0,150.0,0,...,0,0,0,0,0,0,0,0,0,0
1,4,140200,2012,60500,10,2,0,1.5,80.0,0,...,0,0,0,1,1,0,0,0,0,0
2,18,750000,2011,29000,0,0,0,1.3,85.0,0,...,0,0,0,1,1,0,0,0,0,0
3,1,970000,2014,49500,2,0,0,2.0,150.0,0,...,0,0,0,0,1,0,0,0,0,0
4,3,205000,2007,151445,10,2,0,1.4,95.0,0,...,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70107,16,310000,2011,165000,15,2,0,1.2,70.0,0,...,0,0,0,0,1,0,0,0,0,0
70108,5,400000,2011,125000,15,2,0,1.6,105.0,0,...,0,0,0,1,0,0,0,0,0,0
70109,9,1140000,2013,89981,2,0,0,2.0,150.0,0,...,0,0,0,1,0,0,0,0,0,0
70110,19,550550,2006,183000,7,0,0,1.3,92.0,0,...,0,0,0,1,0,0,0,0,0,0


In [ ]:
# Назначение целевой переменной
y = df['price'].values

# Удаление столбца 'price' из данных
df.drop(columns=['price'], inplace=True)

# Назначение обучающих данных
columns = df.columns.tolist()
X = df.values

# Разделение данных на обучающую, валидационную и тестовую выборки
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.1, random_state=42)  # 90% для обучения, 10% для временной выборки
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)  # 5% для валидации и 5% для тестирования

# Вывод шейпов выборок
print("Форма обучающей выборки X_train:", X_train.shape)
print("Форма валидационной выборки X_val:", X_val.shape)
print("Форма тестовой выборки X_test:", X_test.shape)
print("Форма целевой обучающей выборки y_train:", y_train.shape)
print("Форма целевой валидационной выборки y_val:", y_val.shape)
print("Форма целевой тестовой выборки y_test:", y_test.shape)

Форма обучающей выборки X_train: (63100, 1008)
Форма валидационной выборки X_val: (3506, 1008)
Форма тестовой выборки X_test: (3506, 1008)
Форма целевой обучающей выборки y_train: (63100,)
Форма целевой валидационной выборки y_val: (3506,)
Форма целевой тестовой выборки y_test: (3506,)


In [ ]:
# Обучение модели
model = train_model(
    X_train,  # Обучающая выборка
    y_train,  # Целевая переменная для обучения
    X_val,  # Валидационная выборка
    y_val,  # Целевая переменная для валидации
    max_models=20,  # Количество моделей
    seed=42,  # Зерно для воспроизводимости
    max_runtime_secs=1800,  # Максимальное время выполнения в секундах
    stopping_metric='mae'  # Метрика остановки
)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
AutoML progress: |███████████████████████████████████████████████████████████████| (done) 100%


In [ ]:
# Оценка модели
y_pred, mae = evaluate_model(model, X_test, y_test)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
xgboost prediction progress: |███████████████████████████████████████████████████| (done) 100%
Средняя абсолютная ошибка на тестовой выборке: 71366 рублей.
Процент ошибки относительно средней цены: 13.56%


#### **Эксперимент 6** (числовые ячейки: year, mileage, volume, power категориальные ячейки: mark, body, kpp, fuel тестовая ячейка: model + нормализация данных)

In [ ]:
# Запускаем функцию предобработки данных + добавляем вектор текста длинной 1000 параметров
df, encoders, vectorizer = preprocess_data(df)
df

,mark_,price,year,mileage,body,kpp,fuel,volume,power,100,...,z52,zafira,класс,поколение,рестайлинг,с23,с3,серия,сс,х80
0,8,996000,2018,28000,10,0,0,2.0,150.0,0,...,0,0,0,0,0,0,0,0,0,0
1,4,140200,2012,60500,10,2,0,1.5,80.0,0,...,0,0,0,1,1,0,0,0,0,0
2,18,750000,2011,29000,0,0,0,1.3,85.0,0,...,0,0,0,1,1,0,0,0,0,0
3,1,970000,2014,49500,2,0,0,2.0,150.0,0,...,0,0,0,0,1,0,0,0,0,0
4,3,205000,2007,151445,10,2,0,1.4,95.0,0,...,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70107,16,310000,2011,165000,15,2,0,1.2,70.0,0,...,0,0,0,0,1,0,0,0,0,0
70108,5,400000,2011,125000,15,2,0,1.6,105.0,0,...,0,0,0,1,0,0,0,0,0,0
70109,9,1140000,2013,89981,2,0,0,2.0,150.0,0,...,0,0,0,1,0,0,0,0,0,0
70110,19,550550,2006,183000,7,0,0,1.3,92.0,0,...,0,0,0,1,0,0,0,0,0,0


In [ ]:
# Нормализуем данные
columns_to_normalize = ['year', 'mileage', 'volume', 'power', 'mark_', 'body', 'kpp', 'fuel']
df, scalers, columns = prepare_and_normalize_data(df, 'price', columns_to_normalize)
df

,mark_,year,mileage,body,kpp,fuel,volume,power,100,107,...,zafira,класс,поколение,рестайлинг,с23,с3,серия,сс,х80,price
0,0.40,0.985714,0.027999,0.666667,0.000000,0.0,0.175258,0.100072,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.040026
1,0.20,0.900000,0.060499,0.666667,0.666667,0.0,0.123711,0.049676,0.0,0.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.005073
2,0.90,0.885714,0.028999,0.000000,0.000000,0.0,0.103093,0.053276,0.0,0.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.029979
3,0.05,0.928571,0.049499,0.133333,0.000000,0.0,0.175258,0.100072,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.038964
4,0.15,0.828571,0.151444,0.666667,0.666667,0.0,0.113402,0.060475,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.007719
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70107,0.80,0.885714,0.164999,1.000000,0.666667,0.0,0.092784,0.042477,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.012008
70108,0.25,0.885714,0.124999,1.000000,0.666667,0.0,0.134021,0.067675,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.015684
70109,0.45,0.914286,0.089980,0.133333,0.000000,0.0,0.175258,0.100072,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.045908
70110,0.95,0.814286,0.182999,0.466667,0.000000,0.0,0.103093,0.058315,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.021833


In [ ]:
# Назначение обучающих данных и целевой переменной
X = df[columns].values
y = df['price'].values

# Разделение данных на обучающую, валидационную и тестовую выборки
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.1, random_state=42)  # 90% для обучения, 10% для временной выборки
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)  # 5% для валидации и 5% для тестирования

# Вывод шейпов выборок
print("Форма обучающей выборки X_train:", X_train.shape)
print("Форма валидационной выборки X_val:", X_val.shape)
print("Форма тестовой выборки X_test:", X_test.shape)
print("Форма целевой обучающей выборки y_train:", y_train.shape)
print("Форма целевой валидационной выборки y_val:", y_val.shape)
print("Форма целевой тестовой выборки y_test:", y_test.shape)

Форма обучающей выборки X_train: (63100, 1008)
Форма валидационной выборки X_val: (3506, 1008)
Форма тестовой выборки X_test: (3506, 1008)
Форма целевой обучающей выборки y_train: (63100,)
Форма целевой валидационной выборки y_val: (3506,)
Форма целевой тестовой выборки y_test: (3506,)


In [ ]:
# Обучение модели
model = train_model(
    X_train,  # Обучающая выборка
    y_train,  # Целевая переменная для обучения
    X_val,  # Валидационная выборка
    y_val,  # Целевая переменная для валидации
    max_models=20,  # Количество моделей
    seed=42,  # Зерно для воспроизводимости
    max_runtime_secs=1800,  # Максимальное время выполнения в секундах
    stopping_metric='mae'  # Метрика остановки
)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
AutoML progress: |███████████████████████████████████████████████████████████████| (done) 100%


In [ ]:
# Оценка модели
y_pred, mae = evaluate_model_normalized(model, X_test, y_test, scalers['price'])

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
drf prediction progress: |███████████████████████████████████████████████████████| (done) 100%
Средняя абсолютная ошибка на тестовой выборке: 65919 рублей.
Процент ошибки относительно средней цены: 12.53%


In [ ]:
# @title Задать вопрос нейро-куратору

question = "Какие есть способы повысить точность предсказания нейронной сети при решении задач регрессии?" #@param {type:"string"}
url = "http://31.129.102.77:80/curatorv1/qa"
headers = {"Content-Type": "application/json",
    "Authorization": os.environ["UII_API_KEY"]
    }
data = {
    "question": question,
    "user_id": user_id_seed,
    "topic": ''
}

response = requests.post(url, headers=headers, data=json.dumps(data))
print(json.loads(response.text))

Для повышения точности предсказания нейронной сети при решении задач регрессии можно использовать следующие способы:

1. **Использование более сложных моделей**: Попробуйте использовать более глубокие нейронные сети или архитектуры, которые могут лучше улавливать сложные зависимости в данных.

2. **Настройка гиперпараметров**: Экспериментируйте с различными значениями гиперпараметров, такими как learning rate, количество эпох обучения, размер пакета (batch size), функции активации и т. д. 

3. **Регуляризация**: Добавление регуляризации, такой как L1 или L2 регуляризация, может помочь уменьшить переобучение и улучшить обобщающую способность модели.

4. **Использование оптимизаторов**: Выбор подходящего оптимизатора (например, Adam, SGD, RMSprop) также может повлиять на точность предсказаний.

5. **Обработка данных**: Проведите анализ данных, возможно, стоит провести предварительную обработку данных, нормализацию, шкалирование или кодирование категориальных признаков.

6. **Увеличение о

# **2. Задача регрессии: Предсказание суммарных очков в баскетбольном матче**

In [ ]:
#@title Загрузка необходимых библиотек (ЗАПУСТИТЕ ЭТОТ КОД)
!pip install autokeras
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_extraction.text import CountVectorizer
import autokeras as ak
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error, mean_squared_error
from IPython.display import clear_output
clear_output()

In [ ]:
#@title Загрузка сервисных функций (ЗАПУСТИТЕ ЭТОТ КОД)
#@title Функции загрузки данных (ЗАПУСТИТЕ ЭТОТ КОД)
def load_and_preprocess_data(url):
    df = pd.read_csv(url, delimiter=";", encoding="cp1251")
    df.drop_duplicates(inplace=True)
    df.dropna(inplace=True)

    df['TOTAL'] = df['TOTAL'].str.replace(',', '.').astype(float)
    df['Ком. 1'] = df['Ком. 1'].astype(float)
    df['Ком. 2'] = df['Ком. 2'].astype(float)
    df['ftime'] = df['ftime'].astype(float)
    df['fcount'] = df['fcount'].astype(float)

    # Проверка на наличие NaN значений и их удаление
    df = df.dropna()

    return df

#@title Функция нормализации данных (ЗАПУСТИТЕ ЭТОТ КОД)
def normalize_data(X_train, X_val, X_test, y_train, y_val, y_test):
    scalers = {}

    # Нормализация признаков
    for i in range(X_train.shape[1]):
        scalers[i] = MinMaxScaler()
        X_train[:, i] = scalers[i].fit_transform(X_train[:, i].reshape(-1, 1)).flatten()
        X_val[:, i] = scalers[i].transform(X_val[:, i].reshape(-1, 1)).flatten()
        X_test[:, i] = scalers[i].transform(X_test[:, i].reshape(-1, 1)).flatten()

    # Нормализация целевых значений
    y_scaler = MinMaxScaler()
    y_train = y_scaler.fit_transform(y_train.reshape(-1, 1)).flatten()
    y_val = y_scaler.transform(y_val.reshape(-1, 1)).flatten()
    y_test = y_scaler.transform(y_test.reshape(-1, 1)).flatten()

    return X_train, X_val, X_test, y_train, y_val, y_test, y_scaler

#@title Функция для создания и обучения модели (ЗАПУСТИТЕ ЭТОТ КОД)
def train_model(X_train, y_train, X_val, y_val, max_trials, tuner, batch_size, epochs):
    input_node = ak.Input()
    processed_input = ak.DenseBlock()(input_node)
    output_node = ak.RegressionHead(metrics=['mae'])(processed_input)
    auto_model = ak.AutoModel(
        inputs=input_node,
        outputs=output_node,
        max_trials=max_trials,  # Максимальное количество триалов для поиска наилучшей модели
        overwrite=True,
        tuner=tuner  # Тип тюнера (жадный алгоритм)
    )
    auto_model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=epochs, batch_size=batch_size)
    return auto_model

#@title Функция для преобразования текстовых данных в числовые (ЗАПУСТИТЕ ЭТОТ КОД)
def encode_text_data(df, column, max_features=1000):
    # Сначала считаем количество уникальных слов без ограничения
    vectorizer_full = CountVectorizer()
    vectorizer_full.fit(df[column])
    unique_words = len(vectorizer_full.vocabulary_)
    print(f"Реальное количество уникальных слов в текстовых данных: {unique_words}")

    # Затем преобразуем данные с ограничением на количество слов
    vectorizer = CountVectorizer(max_features=max_features)
    encoded = vectorizer.fit_transform(df[column]).toarray()
    encoded_df = pd.DataFrame(encoded, columns=vectorizer.get_feature_names_out())

    return encoded_df

#@title Функция для предсказания и оценки модели (ЗАПУСТИТЕ ЭТОТ КОД)
def evaluate_model(model, X_test, y_test):
    y_pred = model.predict(X_test).squeeze()
    mae = mean_absolute_error(y_test, y_pred)
    average_score = y_test.mean()
    percentage_error = (mae / average_score) * 100
    print(f"Средняя абсолютная ошибка на тестовой выборке: {int(mae)} очков.")
    print(f"Процент ошибки относительно средней оценки: {percentage_error:.2f}%")
    return y_pred, mae

#@title Функция для предсказания и оценки нормализованной модели (ЗАПУСТИТЕ ЭТОТ КОД)
def evaluate_model_normalized(model, X_test, y_test, y_scaler):
    y_pred = model.predict(X_test).squeeze()
    y_test = y_scaler.inverse_transform(y_test.reshape(-1, 1)).flatten()
    y_pred = y_scaler.inverse_transform(y_pred.reshape(-1, 1)).flatten()
    mae = mean_absolute_error(y_test, y_pred)
    average_score = y_test.mean()
    percentage_error = (mae / average_score) * 100
    print(f"Средняя абсолютная ошибка на тестовой выборке: {int(mae)} очков.")
    print(f"Процент ошибки относительно средней оценки: {percentage_error:.2f}%")
    return y_pred, mae

**Описание данных**

В этой таблице содержатся данные о ходе развития 435 баскетбольных матчей. Таблица состоит из следующих колонок:

- **Unnamed: 0**: Индекс строки.
- **TOTAL**: Полный счет, предсказанный букмекерской конторой.
- **info**: Текстовая информация о матче (место проведения, лига).
- **Ком. 1**: Идентификатор первой команды.
- **Ком. 2**: Идентификатор второй команды.
- **Минута**: Минута с начала периода (4 периода по 12 мин).
- **Общая минута**: Общая минута игры.
- **Секунда**: Секунда с начала игры.
- **fcount**: Полный счет игры.
- **ftime**: Полное время в секундах с начала игры.


In [ ]:
# Загружаем данные
url = "https://storage.yandexcloud.net/aiueducation/Content/base/l10/basketball.csv"
df = load_and_preprocess_data(url)
df

,Unnamed: 0,TOTAL,info,Ком. 1,Ком. 2,Минута,Общая минута,Секунда,fcount,ftime
0,0,98.5,4081445 Новая Зеландия. Женщины. WBC. Регулярн...,2.0,0.0,1,1.0,30,81.0,90.0
1,1,100.5,4081445 Новая Зеландия. Женщины. WBC. Регулярн...,2.0,2.0,1,1.0,45,81.0,105.0
2,2,99.5,4081445 Новая Зеландия. Женщины. WBC. Регулярн...,2.0,2.0,2,2.0,0,81.0,120.0
3,3,98.5,4081445 Новая Зеландия. Женщины. WBC. Регулярн...,2.0,2.0,2,2.0,30,81.0,150.0
4,4,95.5,4081445 Новая Зеландия. Женщины. WBC. Регулярн...,2.0,2.0,3,3.0,0,81.0,180.0
...,...,...,...,...,...,...,...,...,...,...
52445,68310,0.0,4163481 Португалия. Мужчины. LPB. Плей-офф. 1/...,88.0,77.0,9,39.0,15,170.0,2355.0
52446,68311,0.0,4163481 Португалия. Мужчины. LPB. Плей-офф. 1/...,88.0,77.0,9,39.0,0,170.0,2340.0
52447,68312,0.0,4163481 Португалия. Мужчины. LPB. Плей-офф. 1/...,90.0,77.0,9,39.0,0,170.0,2340.0
52448,68313,0.0,4163481 Португалия. Мужчины. LPB. Плей-офф. 1/...,90.0,77.0,9,39.0,30,170.0,2370.0


#### **ЭКСПЕРИМЕНТ 1 (используем три числовые ячейки: Ком.1, Ком.2, ftime)**

In [ ]:
# Удаляем столбцы не учавствующие в обучении модели
df.drop(columns=['Unnamed: 0', 'TOTAL', 'info', 'Минута', 'Общая минута', 'Секунда'], inplace=True)
df

,Ком. 1,Ком. 2,fcount,ftime
0,2.0,0.0,81.0,90.0
1,2.0,2.0,81.0,105.0
2,2.0,2.0,81.0,120.0
3,2.0,2.0,81.0,150.0
4,2.0,2.0,81.0,180.0
...,...,...,...,...
52445,88.0,77.0,170.0,2355.0
52446,88.0,77.0,170.0,2340.0
52447,90.0,77.0,170.0,2340.0
52448,90.0,77.0,170.0,2370.0


In [ ]:
# Определяем столбцы для обучения и целевой переменной
columns = ['Ком. 1', 'Ком. 2', 'ftime']
X = df[columns].to_numpy()
y = df['fcount'].to_numpy()

# Разделение данных на обучающую, валидационную и тестовую выборки
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.1, random_state=42)  # 90% для обучения, 10% для временной выборки
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)  # 5% для валидации и 5% для тестирования

# Вывод формы выборок
print("Форма обучающей выборки X_train:", X_train.shape)
print("Форма валидационной выборки X_val:", X_val.shape)
print("Форма тестовой выборки X_test:", X_test.shape)
print("Форма целевой обучающей выборки y_train:", y_train.shape)
print("Форма целевой валидационной выборки y_val:", y_val.shape)
print("Форма целевой тестовой выборки y_test:", y_test.shape)

Форма обучающей выборки X_train: (47205, 3)
Форма валидационной выборки X_val: (2622, 3)
Форма тестовой выборки X_test: (2623, 3)
Форма целевой обучающей выборки y_train: (47205,)
Форма целевой валидационной выборки y_val: (2622,)
Форма целевой тестовой выборки y_test: (2623,)


In [ ]:
# Обучение модели
model = train_model(
    X_train,  # Обучающая выборка
    y_train,  # Целевая переменная для обучения
    X_val,  # Валидационная выборка
    y_val,  # Целевая переменная для валидации
    max_trials=10,  # Количество триалов
    tuner='greedy',  # Тип тюнера
    batch_size=32,  # Размер батча
    epochs=25  # Количество эпох
)

Trial 10 Complete [00h 01m 12s]
val_loss: 344.8808898925781

Best val_loss So Far: 321.34271240234375
Total elapsed time: 00h 13m 53s
Epoch 1/25
1476/1476 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - loss: 19345.1816 - mae: 105.5241 - val_loss: 5992.7251 - val_mae: 62.2629
Epoch 2/25
1476/1476 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 7031.4585 - mae: 71.0317 - val_loss: 4785.8862 - val_mae: 55.7505
Epoch 3/25
1476/1476 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - loss: 5706.9214 - mae: 63.4811 - val_loss: 2589.1436 - val_mae: 41.2083
Epoch 4/25
1476/1476 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 3192.7256 - mae: 46.1459 - val_loss: 674.8088 - val_mae: 20.4904
Epoch 5/25
1476/1476 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 2002.4122 - mae: 35.6495 - val_loss: 619.5630 - val_mae: 19.6917
Epoch 6/25
1476/1476 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - loss: 1893.3093 - mae: 34.5726 - val_loss: 730.5475 - val_mae: 22.3806
Epoch 7/25
1476/1476 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - loss: 1773.8706 - mae: 33.5008 - val_loss: 

In [ ]:
# Оценка модели
y_pred, mae = evaluate_model(model, X_test, y_test)

25/82 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 

/usr/local/lib/python3.10/dist-packages/keras/src/saving/saving_lib.py:415: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 14 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


82/82 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
82/82 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Средняя абсолютная ошибка на тестовой выборке: 12 очков.
Процент ошибки относительно средней оценки: 8.12%


#### **ЭКСПЕРИМЕНТ 2 (используем четыре числовые ячейки: TOTAL, Ком.1, Ком.2, ftime с нормализацией)**

In [ ]:
# Удаляем столбцы не учавствующие в обучении модели
df.drop(columns=['Unnamed: 0', 'info', 'Минута', 'Общая минута', 'Секунда'], inplace=True)
df

,TOTAL,Ком. 1,Ком. 2,fcount,ftime
0,98.5,2.0,0.0,81.0,90.0
1,100.5,2.0,2.0,81.0,105.0
2,99.5,2.0,2.0,81.0,120.0
3,98.5,2.0,2.0,81.0,150.0
4,95.5,2.0,2.0,81.0,180.0
...,...,...,...,...,...
52445,0.0,88.0,77.0,170.0,2355.0
52446,0.0,88.0,77.0,170.0,2340.0
52447,0.0,90.0,77.0,170.0,2340.0
52448,0.0,90.0,77.0,170.0,2370.0


In [ ]:
# Определяем столбцы для обучения и целевой переменной
columns = ['TOTAL', 'Ком. 1', 'Ком. 2', 'ftime']
X = df[columns].to_numpy()
y = df['fcount'].to_numpy()

# Разделение данных на обучающую, валидационную и тестовую выборки
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.1, random_state=42)  # 90% для обучения, 10% для временной выборки
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)  # 5% для валидации и 5% для тестирования

# Вывод формы выборок
print("Форма обучающей выборки X_train:", X_train.shape)
print("Форма валидационной выборки X_val:", X_val.shape)
print("Форма тестовой выборки X_test:", X_test.shape)
print("Форма целевой обучающей выборки y_train:", y_train.shape)
print("Форма целевой валидационной выборки y_val:", y_val.shape)
print("Форма целевой тестовой выборки y_test:", y_test.shape)

Форма обучающей выборки X_train: (47205, 4)
Форма валидационной выборки X_val: (2622, 4)
Форма тестовой выборки X_test: (2623, 4)
Форма целевой обучающей выборки y_train: (47205,)
Форма целевой валидационной выборки y_val: (2622,)
Форма целевой тестовой выборки y_test: (2623,)


In [ ]:
# Нормализация данных
X_train, X_val, X_test, y_train, y_val, y_test, y_scaler = normalize_data(X_train, X_val, X_test, y_train, y_val, y_test)

In [ ]:
# Обучение модели
model = train_model(
    X_train,  # Обучающая выборка
    y_train,  # Целевая переменная для обучения
    X_val,  # Валидационная выборка
    y_val,  # Целевая переменная для валидации
    max_trials=10,  # Количество триалов
    tuner='greedy',  # Тип тюнера
    batch_size=32,  # Размер батча
    epochs=25  # Количество эпох
)

Trial 10 Complete [00h 00m 52s]
val_loss: 0.007238410413265228

Best val_loss So Far: 0.0034445261117070913
Total elapsed time: 00h 09m 35s
Epoch 1/25
1476/1476 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - loss: 0.0208 - mae: 0.0954 - val_loss: 0.0035 - val_mae: 0.0383
Epoch 2/25
1476/1476 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - loss: 0.0049 - mae: 0.0507 - val_loss: 0.0035 - val_mae: 0.0383
Epoch 3/25
1476/1476 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.0039 - mae: 0.0437 - val_loss: 0.0035 - val_mae: 0.0387
Epoch 4/25
1476/1476 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.0036 - mae: 0.0411 - val_loss: 0.0035 - val_mae: 0.0380
Epoch 5/25
1476/1476 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - loss: 0.0035 - mae: 0.0405 - val_loss: 0.0035 - val_mae: 0.0376
Epoch 6/25
1476/1476 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - loss: 0.0035 - mae: 0.0402 - val_loss: 0.0035 - val_mae: 0.0378
Epoch 7/25
1476/1476 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - loss: 0.0035 - mae: 0.0404 - val_loss: 0.0035 - val_mae: 0.0377
Epoch 8/25
1476/1476

In [ ]:
# Оценка нормализованной модели
y_pred, mae = evaluate_model_normalized(model, X_test, y_test, y_scaler)

19/82 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 

/usr/local/lib/python3.10/dist-packages/keras/src/saving/saving_lib.py:415: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 10 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


82/82 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
82/82 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
Средняя абсолютная ошибка на тестовой выборке: 10 очков.
Процент ошибки относительно средней оценки: 6.72%


#### **ЭКСПЕРИМЕНТ 3 (используем четыре числовые ячейки TOTAL, Ком.1, Ком.2, ftime c нормализацией + текстовая ячейка info)**

In [ ]:
# Преобразование текстовых данных в числовые
info_df = encode_text_data(df, 'info', max_features=1000)

Реальное количество уникальных слов в текстовых данных: 1509


In [ ]:
# Удаление столбцов, не участвующих в обучении модели (info удаляем поскольку вместо него добавим вектор)
df.drop(columns=['Unnamed: 0', 'info', 'Минута', 'Общая минута', 'Секунда'], inplace=True)
df

,TOTAL,Ком. 1,Ком. 2,fcount,ftime
0,98.5,2.0,0.0,81.0,90.0
1,100.5,2.0,2.0,81.0,105.0
2,99.5,2.0,2.0,81.0,120.0
3,98.5,2.0,2.0,81.0,150.0
4,95.5,2.0,2.0,81.0,180.0
...,...,...,...,...,...
52445,0.0,88.0,77.0,170.0,2355.0
52446,0.0,88.0,77.0,170.0,2340.0
52447,0.0,90.0,77.0,170.0,2340.0
52448,0.0,90.0,77.0,170.0,2370.0


In [ ]:
# Объединение числовых и текстовых данных и определение столбцов для обучения и целевой переменной
X = pd.concat([df[['TOTAL', 'Ком. 1', 'Ком. 2', 'ftime']], info_df], axis=1).to_numpy()
y = df['fcount'].to_numpy()

# Разделение данных на обучающую, валидационную и тестовую выборки
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.1, random_state=42)  # 90% для обучения, 10% для временной выборки
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)  # 5% для валидации и 5% для тестирования

# Вывод формы выборок
print("Форма обучающей выборки X_train:", X_train.shape)
print("Форма валидационной выборки X_val:", X_val.shape)
print("Форма тестовой выборки X_test:", X_test.shape)
print("Форма целевой обучающей выборки y_train:", y_train.shape)
print("Форма целевой валидационной выборки y_val:", y_val.shape)
print("Форма целевой тестовой выборки y_test:", y_test.shape)

Форма обучающей выборки X_train: (47205, 1004)
Форма валидационной выборки X_val: (2622, 1004)
Форма тестовой выборки X_test: (2623, 1004)
Форма целевой обучающей выборки y_train: (47205,)
Форма целевой валидационной выборки y_val: (2622,)
Форма целевой тестовой выборки y_test: (2623,)


In [ ]:
# Нормализация данных
X_train, X_val, X_test, y_train, y_val, y_test, y_scaler = normalize_data(X_train, X_val, X_test, y_train, y_val, y_test)

In [ ]:
# Обучение модели
model = train_model(
    X_train,  # Обучающая выборка
    y_train,  # Целевая переменная для обучения
    X_val,  # Валидационная выборка
    y_val,  # Целевая переменная для валидации
    max_trials=10,  # Количество триалов
    tuner='greedy',  # Тип тюнера
    batch_size=32,  # Размер батча
    epochs=25  # Количество эпох
)

Trial 10 Complete [00h 03m 11s]
val_loss: 0.00432602921500802

Best val_loss So Far: 8.681822509970516e-05
Total elapsed time: 00h 23m 50s
Epoch 1/25
1476/1476 ━━━━━━━━━━━━━━━━━━━━ 7s 3ms/step - loss: 0.1589 - mae: 0.3520 - val_loss: 0.0105 - val_mae: 0.0773
Epoch 2/25
1476/1476 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - loss: 0.0079 - mae: 0.0665 - val_loss: 0.0047 - val_mae: 0.0483
Epoch 3/25
1476/1476 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - loss: 0.0038 - mae: 0.0430 - val_loss: 0.0028 - val_mae: 0.0340
Epoch 4/25
1476/1476 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - loss: 0.0023 - mae: 0.0306 - val_loss: 0.0018 - val_mae: 0.0250
Epoch 5/25
1476/1476 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - loss: 0.0015 - mae: 0.0224 - val_loss: 0.0013 - val_mae: 0.0185
Epoch 6/25
1476/1476 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - loss: 9.9960e-04 - mae: 0.0166 - val_loss: 8.6524e-04 - val_mae: 0.0138
Epoch 7/25
1476/1476 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - loss: 6.9096e-04 - mae: 0.0125 - val_loss: 6.1817e-04 - val_mae: 0.0108
Epoch

In [ ]:
# Оценка нормализованной модели
y_pred, mae = evaluate_model_normalized(model, X_test, y_test, y_scaler)

/usr/local/lib/python3.10/dist-packages/keras/src/saving/saving_lib.py:415: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 14 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


82/82 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step
82/82 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step
Средняя абсолютная ошибка на тестовой выборке: 1 очков.
Процент ошибки относительно средней оценки: 1.13%


In [ ]:
# @title Задать вопрос нейро-куратору

question = "Как параметризация данных может помочь при решении задач регрессии?" #@param {type:"string"}
url = "http://31.129.102.77:80/curatorv1/qa"
headers = {"Content-Type": "application/json",
    "Authorization": os.environ["UII_API_KEY"]
    }
data = {
    "question": question,
    "user_id": user_id_seed,
    "topic": ''
}

response = requests.post(url, headers=headers, data=json.dumps(data))
print(json.loads(response.text))

Параметризация данных играет важную роль при решении задач регрессии. Позволь мне разложить это по шагам:

1. **Нормализация данных**: Перед тем, как приступить к обучению модели регрессии, данные часто нормализуются. Нормализация помогает привести все признаки к одному масштабу, что помогает модели быстрее сходиться и избежать проблем с градиентом.

2. **Преобразование категориальных данных**: Если в данных присутствуют категориальные признаки, их нужно преобразовать в числовой формат. Это может быть сделано с помощью методов, таких как One-Hot Encoding или Label Encoding.

3. **Обработка пропущенных значений**: При наличии пропущенных значений в данных, их необходимо обработать. Это может включать заполнение средними значениями, медианами или удаление соответствующих строк.

4. **Инженерия признаков**: Создание новых признаков на основе существующих данных может улучшить производительность модели. Например, можно добавить полиномиальные признаки или комбинации существующих признаков.

#  **3. Задача классификации: Предсказание сердечных заболеваний по анкетным данным**

Данные о медицинских обследованиях предоставлены **Комитетом цифрового развития и связи Курской области** для  чемпионата **«цифрового прорыва»** в *Курской области*

In [ ]:
#@title Загрузка необходимых библиотек (ЗАПУСТИТЕ ЭТОТ КОД)
!pip install autokeras
import pandas as pd
import numpy as np
import autokeras as ak
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, recall_score, precision_score, classification_report
from IPython.display import clear_output
pd.set_option('display.max_columns', None)
clear_output()
from sklearn.utils import shuffle

In [ ]:
#@title Сервисные функции (ЗАПУСТИТЕ ЭТОТ КОД)
# Функция загрузки данных
def load_data(url):
    df = pd.read_csv(url)
    return df

# Функция преподготовки данных
def preprocess_data(df):
    # Удаление ненужных столбцов, если они существуют
    columns_to_drop = ['ID', 'ID_y']
    df = df.drop(columns=[col for col in columns_to_drop if col in df.columns], errors='ignore')

    # Удаление строк с пустыми значениями в столбце "Пол"
    df.dropna(subset=['Пол'], inplace=True)

    # Определяем целевые столбцы
    target_columns = ['Артериальная гипертензия', 'ОНМК', 'Стенокардия, ИБС, инфаркт миокарда', 'Сердечная недостаточность', 'Прочие заболевания сердца']
    #df = df[(df[target_columns] != 0).any(axis=1)]

    # Замена всех оставшихся значений NaN на 0
    df.fillna(0, inplace=True)

    # Кодирование категориальных признаков
    categorical_columns = ['Пол', 'Семья', 'Этнос', 'Национальность', 'Религия', 'Образование', 'Профессия', 'Статус Курения',
                           'Частота пасс кур', 'Алкоголь', 'Время засыпания', 'Время пробуждения']
    df, encoders = encode_categorical_features(df, categorical_columns)

    # Преобразование всех столбцов в числовой формат
    df = df.apply(pd.to_numeric, errors='coerce')

    # Перемешивание данных
    df = shuffle(df, random_state=163)

    # Отделение последних 50 записей для тестовой выборки
    df_test = df.iloc[-50:]
    df = df.iloc[:-50]

    # Дублирование данных для балансировки классов
    df_balanced = balance_classes(df, target_columns)

    # Перемешивание данных после балансировки
    df_balanced = shuffle(df_balanced, random_state=111)

    # Переиндексация DataFrame
    df_balanced.reset_index(drop=True, inplace=True)

    return df_balanced, df_test

# Функция перевода в категориальны признаки
def encode_categorical_features(df, columns):
    encoders = {}
    for col in columns:
        encoders[col] = LabelEncoder()
        df.loc[:, col] = encoders[col].fit_transform(df[col].astype(str))
    return df, encoders

# Функция баланса классов
def balance_classes(df, target_columns):
    multipliers = {
        'ОНМК': 10,
        'Стенокардия, ИБС, инфаркт миокарда': 4,
        'Сердечная недостаточность': 4,
        'Прочие заболевания сердца': 5
    }

    balanced_df = df.copy()

    for target in target_columns:
        if target in multipliers:
            multiplier = multipliers[target]
            df_target = df[df[target] == 1]
            balanced_df = pd.concat([balanced_df] + [df_target] * (multiplier - 1), ignore_index=True)

    return balanced_df

# Функция разделения данных на обучающие и валидационные выборки
def split_data(df, features, target_columns):
    X = df[features].values
    y = df[target_columns].values

    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=52)

    return X_train, X_val, y_train, y_val

# Функция создания и обучения модели
def train_model(auto_model, X_train, y_train, X_val, y_val, epochs):
    auto_model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=epochs)
    return auto_model

# Функция оценки модели
def evaluate_model(model, X_test, y_test, target_columns, threshold=0.9):
    predicted_y = model.predict(X_test)
    predicted_classes = (predicted_y > threshold).astype(int)

    # Вычисление precision, recall и support (количество каждого класса)
    report = classification_report(y_test, predicted_classes, target_names=target_columns, zero_division=0, output_dict=True)

    precision_scores = {class_name: report[class_name]['precision'] for class_name in target_columns}
    recall_scores = {class_name: report[class_name]['recall'] for class_name in target_columns}
    support_scores = {class_name: report[class_name]['support'] for class_name in target_columns}

    # Вывод результатов
    print("Результаты на тестовой выборке для каждого класса:")
    for class_name in target_columns:
        support = support_scores[class_name]
        precision = precision_scores[class_name]
        recall = recall_scores[class_name]
        print(f"{class_name}: Количество = {support}, Precision = {precision:.2f}, Recall = {recall:.2f}")

**ПАРАМЕТРЫ ДЛЯ ОБУЧЕНИЯ МОДЕЛИ:**


**Пол**: Бинарный столбец, указывающий на пол человека

**Семья**: Категориальный столбец, указывающий на семейное положение человека

**Этнос**: Категориальный столбец, указывающий на этническую принадлежность

**Национальность**: Категориальный столбец, указывающий на национальность человека.

**Религия**: Категориальный столбец, указывающий на религиозную принадлежность человека.

**Образование**: Категориальный столбец, указывающий на уровень образования человека.

**Профессия**: Категориальный столбец, указывающий на профессию человека.

**Вы работаете?**: Бинарный столбец, указывающий, работает ли человек в настоящее время.

**Выход на пенсию**: Бинарный столбец, указывающий, вышел ли человек на пенсию.

**Прекращение работы по болезни**: Бинарный столбец, указывающий, прекратил ли человек работать из-за болезни.

**Сахарный диабет**: Бинарный столбец, указывающий на наличие сахарного диабета.

**Гепатит**: Бинарный столбец, указывающий на наличие гепатита.

**Онкология**: Бинарный столбец, указывающий на наличие онкологического заболевания.

**Хроническое заболевание легких**: Бинарный столбец, указывающий на наличие хронического заболевания легких.

**Бронжиальная астма**: Бинарный столбец, указывающий на наличие бронхиальной астмы.

**Туберкулез легких**: Бинарный столбец, указывающий на наличие туберкулеза легких.

**ВИЧ/СПИД**: Бинарный столбец, указывающий на наличие ВИЧ/СПИД.

**Регулярный прим лекарственных средств**: Бинарный столбец, указывающий, принимает ли человек регулярно лекарственные средства.

**Травмы за год**: Бинарный столбец, указывающий, были ли у человека травмы за год.

**Переломы**: Бинарный столбец, указывающий, были ли у человека переломы.

**Статус Курения**: Категориальный столбец, указывающий на статус курения человека.

**Возраст курения**: Числовой столбец, указывающий на возраст начала курения.

**Сигарет в день**: Числовой столбец, указывающий на количество сигарет в день.

**Пассивное курение**: Бинарный столбец, указывающий, подвергается ли человек пассивному курению.

**Частота пасс кур**: Категориальный столбец, указывающий на частоту пассивного курения.

**Алкоголь**: Категориальный столбец, указывающий на употребление алкоголя человеком.

**Возраст алког**: Числовой столбец, указывающий на возраст начала употребления алкоголя.

**Время засыпания**: Временной столбец, указывающий на время засыпания человека.

**Время пробуждения**: Временной столбец, указывающий на время пробуждения человека.

**Сон после обеда**: Бинарный столбец, указывающий, спит ли человек после обеда.

**Спорт, клубы**: Бинарный столбец, указывающий, занимается ли человек спортом или посещает клубы.

**Религия, клубы**: Бинарный столбец, указывающий, посещает ли человек религиозные клубы.

**ЦЕЛЕВЫЕ ЗНАЧЕНИЯ ДЛЯ КЛАССИФИКАЦИИ**

**Артериальная гипертензия**: Бинарный столбец, указывающий на наличие артериальной гипертензии

**ОНМК**: Бинарный столбец, указывающий на наличие острых нарушений мозгового кровообращения

**Стенокардия, ИБС, инфаркт миокарда**: Бинарный столбец, указывающий на наличие стенокардии, ИБС или инфаркта миокарда

**Сердечная недостаточность**: Бинарный столбец, указывающий на наличие сердечной недостаточности

**Прочие заболевания сердца**: Бинарный столбец, указывающий на наличие прочих заболеваний сердца

In [ ]:
# Загружаем данные
url = "https://storage.yandexcloud.net/aiueducation/marketing/datasets/medicine.csv"
df = load_data(url)
df

,ID,Пол,Семья,Этнос,Национальность,Религия,Образование,Профессия,Вы работаете?,Выход на пенсию,Прекращение работы по болезни,Сахарный диабет,Гепатит,Онкология,Хроническое заболевание легких,Бронжиальная астма,Туберкулез легких,ВИЧ/СПИД,Регулярный прим лекарственных средств,Травмы за год,Переломы,Статус Курения,Возраст курения,Сигарет в день,Пассивное курение,Частота пасс кур,Алкоголь,Возраст алког,Время засыпания,Время пробуждения,Сон после обеда,"Спорт, клубы","Религия, клубы",ID_y,Артериальная гипертензия,ОНМК,"Стенокардия, ИБС, инфаркт миокарда",Сердечная недостаточность,Прочие заболевания сердца
0,54-102-358-02,М,в браке в настоящее время,европейская,Русские,Христианство,3 - средняя школа / закон.среднее / выше среднего,низкоквалифицированные работники,1,0,0,0,0,0,0,0,0,0,0,0,0,Курит,15.0,20.0,0,NaN,употребляю в настоящее время,18.0,22:00:00,06:00:00,0,0,0,54-102-358-02,0,0,0,0,0
1,54-103-101-01,Ж,в разводе,европейская,Русские,Христианство,5 - ВУЗ,дипломированные специалисты,0,0,0,1,0,0,0,0,0,0,1,0,1,Никогда не курил(а),NaN,NaN,0,NaN,никогда не употреблял,NaN,00:00:00,04:00:00,1,0,0,54-103-101-01,1,1,0,0,0
2,54-501-026-03,Ж,в браке в настоящее время,европейская,Русские,Христианство,5 - ВУЗ,дипломированные специалисты,0,0,0,0,0,0,0,0,0,0,1,0,0,Никогда не курил(а),NaN,NaN,1,1-2 раза в неделю,употребляю в настоящее время,17.0,23:00:00,07:00:00,0,0,0,54-501-026-03,0,0,0,0,0
3,54-501-094-02,М,в браке в настоящее время,европейская,Русские,Атеист / агностик,3 - средняя школа / закон.среднее / выше среднего,низкоквалифицированные работники,1,0,0,0,0,1,0,0,0,0,1,0,0,Бросил(а),12.0,10.0,1,3-6 раз в неделю,употребляю в настоящее время,13.0,23:00:00,07:00:00,0,0,0,54-501-094-02,1,0,0,0,0
4,54-503-022-01,Ж,в браке в настоящее время,европейская,Русские,Христианство,3 - средняя школа / закон.среднее / выше среднего,операторы и монтажники установок и машинного о...,0,0,1,1,1,0,0,0,0,0,1,0,1,Никогда не курил(а),NaN,NaN,1,не менее 1 раза в день,употребляю в настоящее время,16.0,23:00:00,06:00:00,0,0,0,54-503-022-01,1,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
950,54-103-006-01,М,в браке в настоящее время,европейская,Русские,Христианство,4 - профессиональное училище,операторы и монтажники установок и машинного о...,1,0,0,0,0,0,0,0,0,0,0,0,1,Курит,16.0,20.0,0,NaN,употребляю в настоящее время,18.0,22:00:00,06:00:00,0,0,0,54-103-006-01,0,0,0,0,0
951,54-701-039-01,Ж,вдовец / вдова,европейская,Русские,Христианство,3 - средняя школа / закон.среднее / выше среднего,низкоквалифицированные работники,1,0,0,0,0,0,0,0,0,0,1,0,0,Курит,15.0,10.0,0,NaN,ранее употреблял,26.0,21:30:00,07:15:00,0,0,0,54-701-039-01,0,0,0,0,0
952,54-002-014-01,Ж,в браке в настоящее время,европейская,Русские,Христианство,3 - средняя школа / закон.среднее / выше среднего,служащие,0,1,0,0,0,0,0,0,0,0,1,0,0,Бросил(а),20.0,10.0,0,NaN,употребляю в настоящее время,20.0,22:30:00,08:00:00,1,0,0,54-002-014-01,1,0,1,0,0
953,54-501-022-01,Ж,в браке в настоящее время,европейская,Русские,Атеист / агностик,5 - ВУЗ,"представители законодат. органов власти,...",1,0,0,0,1,0,0,0,0,0,1,0,0,Никогда не курил(а),NaN,NaN,0,NaN,употребляю в настоящее время,21.0,23:00:00,08:30:00,0,1,0,54-501-022-01,0,0,0,0,0


In [ ]:
# Предобрабатываем данные:
# 1. Удаляем ненужные столбцы
# 2. Проводим балансировку слабо представленных классов
# 3. Категоризируем данные
df, df_test = preprocess_data(df)
df

,Пол,Семья,Этнос,Национальность,Религия,Образование,Профессия,Вы работаете?,Выход на пенсию,Прекращение работы по болезни,Сахарный диабет,Гепатит,Онкология,Хроническое заболевание легких,Бронжиальная астма,Туберкулез легких,ВИЧ/СПИД,Регулярный прим лекарственных средств,Травмы за год,Переломы,Статус Курения,Возраст курения,Сигарет в день,Пассивное курение,Частота пасс кур,Алкоголь,Возраст алког,Время засыпания,Время пробуждения,Сон после обеда,"Спорт, клубы","Религия, клубы",Артериальная гипертензия,ОНМК,"Стенокардия, ИБС, инфаркт миокарда",Сердечная недостаточность,Прочие заболевания сердца
0,0,0,1,12,3,3,7,1,0,0,0,0,0,0,0,0,0,1,0,0,2,0.0,0.0,0,0,2,25.0,18,28,0,0,0,0,0,0,0,1
1,1,0,1,12,0,2,10,0,1,1,1,0,1,0,0,0,0,1,0,1,0,17.0,40.0,0,0,1,15.0,0,28,1,0,0,0,0,0,0,0
2,0,0,1,12,3,2,7,0,1,0,0,0,0,0,0,0,0,1,0,1,0,40.0,5.0,0,0,2,25.0,0,20,0,0,0,1,0,0,0,1
3,0,0,1,12,3,2,7,1,0,0,0,0,0,0,0,0,0,0,0,0,2,0.0,0.0,0,0,0,0.0,18,20,0,0,0,1,0,0,0,0
4,1,0,1,12,3,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,2,0.0,0.0,1,4,1,21.0,18,11,0,0,0,1,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2164,0,0,1,12,3,3,2,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0.0,0.0,0,0,1,16.0,3,25,1,0,0,0,0,0,0,0
2165,0,0,1,12,3,3,2,0,0,0,0,0,0,0,0,0,0,1,0,0,2,0.0,0.0,0,0,2,35.0,18,11,0,0,0,0,0,0,0,0
2166,0,0,1,12,3,3,6,0,1,0,0,0,1,0,0,0,0,1,0,0,2,0.0,0.0,1,1,2,25.0,15,11,0,0,0,1,0,0,0,0
2167,1,1,1,12,3,2,8,0,1,1,0,0,0,0,0,0,0,1,0,1,1,15.0,20.0,0,0,1,15.0,17,25,0,0,0,1,1,0,0,0


In [ ]:
# Назначаем столбцы для обучения и целевые столбцы
features = ['Пол', 'Семья', 'Этнос', 'Национальность', 'Религия', 'Образование', 'Профессия', 'Вы работаете?', 'Выход на пенсию',
            'Прекращение работы по болезни', 'Сахарный диабет', 'Гепатит', 'Онкология', 'Хроническое заболевание легких', 'Бронжиальная астма', 'Туберкулез легких ',
            'ВИЧ/СПИД', 'Регулярный прим лекарственных средств', 'Травмы за год', 'Переломы', 'Статус Курения', 'Возраст курения',
            'Сигарет в день', 'Пассивное курение', 'Частота пасс кур', 'Алкоголь', 'Возраст алког', 'Время засыпания', 'Время пробуждения',
            'Сон после обеда', 'Спорт, клубы', 'Религия, клубы']
target_columns = ['Артериальная гипертензия', 'ОНМК', 'Стенокардия, ИБС, инфаркт миокарда', 'Сердечная недостаточность', 'Прочие заболевания сердца']

# Разделяем данные на обучающую и проверочную выборку (тестовую выборку сформировали ранее)
X_train, X_val, y_train, y_val = split_data(df, features, target_columns)
X_test = df_test[features].values
y_test = df_test[target_columns].values

# Формы выборок перед подачей в модель
print("\nФорма данных перед подачей в модель:")
print(f"X_train: {X_train.shape}, y_train: {y_train.shape}")
print(f"X_val: {X_val.shape}, y_val: {y_val.shape}")
print(f"X_test: {X_test.shape}, y_test: {y_test.shape}")


Форма данных перед подачей в модель:
X_train: (1952, 32), y_train: (1952, 5)
X_val: (217, 32), y_val: (217, 5)
X_test: (50, 32), y_test: (50, 5)


In [ ]:
# Параметры модели
input_node = ak.Input()
output_node = ak.ClassificationHead()(input_node)

# Создание и обучение модели AutoKeras
auto_model = ak.AutoModel(
    inputs=input_node,
    outputs=output_node,
    max_trials=50,
    overwrite=True
)

# Обучение модели
epochs = 100
model = train_model(auto_model, X_train, y_train, X_val, y_val, epochs)

Trial 14 Complete [00h 00m 20s]
val_loss: 11.617212295532227

Best val_loss So Far: 2.4674875736236572
Total elapsed time: 00h 03m 33s
Epoch 1/100
62/62 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.5560 - loss: 14.4298 - val_accuracy: 0.7534 - val_loss: 6.5718
Epoch 2/100
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.4253 - loss: 8.3305 - val_accuracy: 0.7854 - val_loss: 5.0746
Epoch 3/100
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.4502 - loss: 6.1119 - val_accuracy: 0.8311 - val_loss: 4.1030
Epoch 4/100
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.4716 - loss: 5.2392 - val_accuracy: 0.8265 - val_loss: 3.5633
Epoch 5/100
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.4942 - loss: 4.5191 - val_accuracy: 0.8219 - val_loss: 3.1858
Epoch 6/100
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.4971 - loss: 4.0397 - val_accuracy: 0.8037 - val_loss: 2.9955
Epoch 7/100
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5022 - loss: 3.7416 - val_accuracy: 0

In [ ]:
# Оценка модели
evaluate_model(model, X_test, y_test, target_columns)

/usr/local/lib/python3.10/dist-packages/keras/src/saving/saving_lib.py:415: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 38 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 244ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
Результаты на тестовой выборке для каждого класса:
Артериальная гипертензия: Количество = 20, Precision = 0.70, Recall = 0.35
ОНМК: Количество = 4, Precision = 0.44, Recall = 1.00
Стенокардия, ИБС, инфаркт миокарда: Количество = 4, Precision = 1.00, Recall = 0.25
Сердечная недостаточность: Количество = 5, Precision = 0.43, Recall = 0.60
Прочие заболевания сердца: Количество = 6, Precision = 0.17, Recall = 0.67


**Precision (точность)** — это доля правильных положительных предсказаний среди всех предсказанных положительных случаев.

- Пример: Если модель предсказала 100 случаев артериальной гипертензии, и из них 93 случаев были правильными, то precision будет 93%.

**Recall (полнота)** — это доля правильных положительных предсказаний среди всех истинных положительных случаев.

- Пример: Если у вас есть 100 случаев артериальной гипертензии, и модель правильно предсказала 60 из них, то recall будет 60%.


In [ ]:
# @title Задать вопрос нейро-куратору

question = "Как повысить точность работы нейронной сети при решении задач классификации табличных данных?" #@param {type:"string"}
url = "http://31.129.102.77:80/curatorv1/qa"
headers = {"Content-Type": "application/json",
    "Authorization": os.environ["UII_API_KEY"]
    }
data = {
    "question": question,
    "user_id": user_id_seed,
    "topic": ''
}

response = requests.post(url, headers=headers, data=json.dumps(data))
print(json.loads(response.text))

Для повышения точности работы нейронной сети при решении задач классификации табличных данных можно применить несколько методов и техник. Вот несколько шагов, которые могут помочь улучшить точность:

1. **Предобработка данных**:
   - Изучите и проанализируйте данные, чтобы понять их структуру и особенности.
   - Обработайте отсутствующие значения, выбросы и проведите нормализацию данных.
   - Преобразуйте категориальные признаки в числовой формат, например, с помощью кодирования One-Hot Encoding или Label Encoding.

2. **Выбор и настройка модели**:
   - Выберите подходящую архитектуру нейронной сети для работы с табличными данными. Например, можно использовать многослойный перцептрон (MLP) или глубокие нейронные сети.
   - Настройте гиперпараметры модели, такие как скорость обучения, количество слоев, количество нейронов в каждом слое и функции активации.
   - Используйте регуляризацию, чтобы предотвратить переобучение модели.

3. **Обучение модели**:
   - Разделите данные на обучающий

# **Дополнительные ресурсы**

Для углубленного изучения задач регрессии и классификации табличных данных, рекомендуем следующие источники данных и литературу:

1. [UCI Machine Learning Repository - Tabular Data](https://archive.ics.uci.edu/ml/index.php)
2. [Kaggle Datasets - Tabular Data](https://www.kaggle.com/datasets?search=Tabular+Data)
3. [Kaggle Learn - Pandas](https://www.kaggle.com/learn/pandas)


Эти ресурсы предоставят вам доступ к разнообразным наборам данных и статьям, которые можно использовать для практики и развития навыков в задачах регрессии и классификации табличных данных.

Спасибо за участие в нашем вебинаре, и удачи в ваших будущих исследованиях табличных данных!


# **Самостоятельная работа**

In [ ]:
#@title Вопросы для проверки знаний
!pip install gdown
import pandas as pd
import random
import gdown
from IPython.display import clear_output
clear_output()

# Загрузка файла с Google Drive
url = "https://drive.google.com/uc?id=1kV0n_PNtAMuA75FFFQUDrXvuPHZPgPy-"
output = "AutoKeras_Quiz_Questions_Combined.csv"
gdown.download(url, output, quiet=False, fuzzy=True)

# Загрузка датафрейма с вопросами и ответами
df = pd.read_csv(output)
#print(df)
# Создание списка индексов вопросов
question_indices = list(range(len(df)))

# Перемешивание индексов для случайного выбора
random.shuffle(question_indices)

# Инициализация переменной для подсчета правильных ответов
correct_answers = 0

# Предоставление вопросов и проверка ответов
for idx in question_indices[:6]:
    question = df.loc[idx, 'Вопросы с вариантами ответов']
    correct_answer = df.loc[idx, 'Правильный ответ']
    print(question)
    user_answer = input("Введите ваш ответ: ")

    # Проверка правильности ответа
    if user_answer.lower() == correct_answer.lower():
        correct_answers += 1
        print("Правильно!")
    else:
        print(f"Неправильно. Правильный ответ: {correct_answer}")

# Вывод результата количества правильных ответов и общего количества вопросов
print(f"Правильно {correct_answers} из 6")


Downloading...
From: https://drive.google.com/uc?id=1kV0n_PNtAMuA75FFFQUDrXvuPHZPgPy-
To: /content/AutoKeras_Quiz_Questions_Combined.csv
100%|██████████| 2.70k/2.70k [00:00<00:00, 5.95MB/s]


Что такое классификация в машинном обучении? A) Метод прогнозирования непрерывных значений B) Метод уменьшения размерности C) Метод прогнозирования категориальных меток
Введите ваш ответ: C
Правильно!
Какой алгоритм используется для задачи классификации? A) Линейная регрессия B) K-ближайших соседей C) PCA
Введите ваш ответ: B
Правильно!
Какой метод может быть использован для борьбы с переобучением в моделях регрессии? A) Увеличение размера обучающей выборки B) Добавление большего количества признаков C) Регуляризация
Введите ваш ответ: C
Правильно!
Какая библиотека используется для автоматизации моделирования табличных данных? A) Pandas B) AutoKeras C) Matplotlib
Введите ваш ответ: B
Правильно!
Что такое F1-мера в задачах классификации? A) Среднее значение точности и полноты B) Гармоническое среднее точности и полноты C) Максимальное значение точности и полноты
Введите ваш ответ: B
Правильно!
Что такое регрессия в машинном обучении? A) Метод классификации B) Метод прогнозирования непре

# **Домашнее задание**

Выполните домашнее задание:

- [задание Lite](https://colab.research.google.com/drive/1oTU0oyunduqpvGNYbpP7nSE2XNMbkENG?usp=sharing)
- [задание Pro](https://colab.research.google.com/drive/1WzsXPlWUQb0Qn6nxq8YnMrJzADF0Cmd3?usp=sharing)

и сохраните в файлах с расширением .ipynb

In [ ]:
#@title Выбираем файл с выполненным домашним заданием уровня lite
from google.colab import files

# Загружаем файлы
uploaded = files.upload()

# Перебираем все загруженные файлы
for filename in uploaded.keys():
    print('Файл "{name}" загружен!'.format(name=filename))
    # Открываем файл на чтение
    with open(filename, 'r') as file:
        # Читаем содержимое файла и сохраняем его в переменную
        resheniye = file.read()

Saving Домашнее задание lite_ Нейросети для табличных данных.ipynb to Домашнее задание lite_ Нейросети для табличных данных.ipynb
Файл "Домашнее задание lite_ Нейросети для табличных данных.ipynb" загружен!


In [ ]:
#@title Запуск проверки
import json
import requests

tema = "Нейросети для табличных данных"
dz = '''Задача уровня lite:
Задача: Прогнозирование цены дома с использованием H2O AutoML
Описание задачи: Используя данные о домах, обучите модель для прогнозирования цены на основе различных характеристик дома с использованием H2O AutoML. Оцените качество модели и сделайте выводы.'''

url = "http://62.113.108.33/vebinar-v1/check_dz"
headers = {"Content-Type": "application/json",
    "Authorization": '7128c9c8-eadb-4e39-9b46-372435f6a710'
    }
data = {
    "tema": tema,
    "dz": dz,
    "resheniye": resheniye
}

response = requests.post(url, headers=headers, data=json.dumps(data))
if response.status_code == 200:
  print(json.loads(response.text)['choices'][0]['message']['content'])
else:
  print("Возникла ошибка:", response.status_code)
  print(response.text)

### Оценка решения студента

#### Общие замечания:
1. **Структура и оформление**:
   - Структура ноутбука логична и последовательна.
   - Оформление и комментарии помогают понять ход выполнения задания.

2. **Загрузка библиотек и данных**:
   - Все необходимые библиотеки загружены корректно.
   - Данные загружены и предварительно обработаны.

3. **Разделение данных**:
   - Данные корректно разделены на обучающую, валидационную и тестовую выборки.

4. **Обучение модели**:
   - Модель обучена с использованием H2O AutoML.
   - Параметры обучения (max_models и max_runtime_secs) заданы адекватно.

5. **Оценка модели**:
   - Модель оценена на тестовой выборке.
   - Средняя абсолютная ошибка (MAE) рассчитана и выведена.

#### Детальный анализ:

1. **Загрузка библиотек и инициализация H2O**:
   ```python
   !pip install h2o scikit-learn
   import pandas as pd
   import numpy as np
   import h2o
   from h2o.automl import H2OAutoML
   from sklearn.datasets import fetch_california_housing
   from

In [ ]:
#@title Выбираем файл с выполненным домашним заданием уровня Pro
from google.colab import files

# Загружаем файлы
uploaded = files.upload()

# Перебираем все загруженные файлы
for filename in uploaded.keys():
    print('Файл "{name}" загружен!'.format(name=filename))
    # Открываем файл на чтение
    with open(filename, 'r') as file:
        # Читаем содержимое файла и сохраняем его в переменную
        resheniye = file.read()

Saving Домашнее задание pro_ Нейросети для табличных данных.ipynb to Домашнее задание pro_ Нейросети для табличных данных.ipynb
Файл "Домашнее задание pro_ Нейросети для табличных данных.ipynb" загружен!


In [ ]:
#@title Запуск проверки
import json
import requests

tema = "Домашнее задание: Нейросети для табличных данных"
dz = '''Задача уровня pro:
Задача: Классификация типа цветка Iris с использованием AutoKeras
Описание задачи: Используя данные о цветках Iris, обучите модель для классификации типа цветка (Setosa, Versicolor, Virginica) с использованием AutoKeras. Добейтесь точности 95% и сделайте выводы.'''

url = "http://62.113.108.33/vebinar-v1/check_dz"
headers = {"Content-Type": "application/json",
    "Authorization": '7128c9c8-eadb-4e39-9b46-372435f6a710'
    }
data = {
    "tema": tema,
    "dz": dz,
    "resheniye": resheniye
}

response = requests.post(url, headers=headers, data=json.dumps(data))
if response.status_code == 200:
  print(json.loads(response.text)['choices'][0]['message']['content'])
else:
  print("Возникла ошибка:", response.status_code)
  print(response.text)

### Оценка решения студента

#### Общие замечания
1. **Структура и оформление**:
   - Решение хорошо структурировано, с четким разделением на секции.
   - Комментарии и заголовки помогают понять, что происходит на каждом этапе.

2. **Использование библиотек**:
   - Все необходимые библиотеки загружены и использованы корректно.
   - AutoKeras и scikit-learn применены правильно.

3. **Предобработка данных**:
   - Данные загружены и предобработаны корректно.
   - Нормализация данных выполнена с использованием MinMaxScaler, что является хорошей практикой.

4. **Обучение модели**:
   - Модель AutoKeras настроена и обучена корректно.
   - Использование `max_trials=50` и `epochs=50` показывает, что студент старался найти оптимальную модель.

5. **Оценка модели**:
   - Модель оценена на тестовой выборке, и результаты представлены в виде точности и отчета о классификации.
   - Точность модели 97% является отличным результатом.

6. **Предсказание новых данных**:
   - Пример использования модели 